In [1]:
import cv2
import numpy as np
import os
import sys
import time

module_path = os.path.abspath(os.path.join(r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\poly-yolo-master\poly_yolo'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
#load pretrained model
    
# import poly_yolo_lite as yolo #or "import poly_yolo_lite as yolo" for the lite version
import poly_yolo as yolo #or "import poly_yolo_lite as yolo" for the lite version


#if you want to detect more objects, lower the score and vice versa
# trained_model = yolo.YOLO(model_path=r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\poly-yolo-master\poly_yolo\models\poly_yolo_lite.h5', iou=0.5, score=0.3)
trained_model = yolo.YOLO(model_path=r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\poly-yolo-master\poly_yolo\modelsep035-loss8.233-val_loss7.902.h5', iou=0.5, score=0.3)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
K:\OneDrive_NJIT\OneDrive - NJIT\Research\poly-yolo-master\poly_yolo\modelsep035-loss8.233-val_loss7.902.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
#helper function
def translate_color(cls):
    if cls == 0: return (230, 25, 75)
    if cls == 1: return (60, 180, 75)
    if cls == 2: return (255, 225, 25)
    if cls == 3: return (0, 130, 200)
    if cls == 4: return (245, 130, 48)
    if cls == 5: return (145, 30, 180)
    if cls == 7: return (70, 240, 240)
    if cls == 8: return (240, 50, 230)
    if cls == 9: return (210, 245, 60)
    if cls == 10: return (250, 190, 190)
    if cls == 11: return (0, 128, 128)
    if cls == 12: return (230, 190, 255)
    if cls == 13: return (170, 110, 40)
    if cls == 14: return (255, 250, 200)
    if cls == 15: return (128, 0, 128)
    if cls == 16: return (170, 255, 195)
    if cls == 17: return (128, 128, 0)
    if cls == 18: return (255, 215, 180)
    if cls == 19: return (80, 80, 128)

In [6]:
import glob

dir_imgs_name = r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\poly-yolo-master\simulator_dataset\imgs' #path_where_are_images_to_clasification
out_path       = r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\poly-yolo-master\test_imgs\poly_yolo_predict2' #path, where the images will be saved. The path must exist
list_of_imgs = [root+"\\"+name for root, dirs, files in os.walk(dir_imgs_name) for name in files] 

# list_of_imgs = glob.glob(r'K:\Research\Trees\NewYorkCity_test\google_street_images\*.jpg')

list_of_imgs.sort()

print(len(list_of_imgs))

#browse all images
total_boxes = 0
imgs        = 0
for im in range (0, len(list_of_imgs)):
    imgs    += 1
    img     = cv2.imread(list_of_imgs[im])
    overlay = img.copy()
    boxes   = []
    scores  = []
    classes = []
    
    bname = os.path.basename(list_of_imgs[im])
#     print(bname)
    
    #realize prediciction using poly-yolo
    startx = time.time()
    box, scores, classes, polygons = trained_model.detect_image(img)
    
    print("box:\n", box)
    print("scores:\n", scores)
    print("classes:\n", classes)
    print("polygons:\n", polygons)
    print("len box:\n", len(box))
    print("len polygons[0]:\n", len(polygons[0]))

    
    if imgs % 99 == 0:
        print('Prediction speed: ', 1.0/(time.time() - startx), 'fps')
    
    
    #example, hw to reshape reshape y1,x1,y2,x2 into x1,y1,x2,y2
    for k in range (0, len(box)):
        boxes.append((box[k][1], box[k][0], box[k][3], box[k][2]))
        cv2.rectangle(img, (box[k][1],box[k][0]), (box[k][3],box[k][2]), translate_color(classes[k]), 3, 1)
    total_boxes += len(boxes)
    
    #browse all boxes
    for b in range(0, len(boxes)):
        f              = translate_color(classes[b])    
        points_to_draw = []
        offset         = len(polygons[b])//3
        
        #filter bounding polygon vertices
        for dst in range(0, len(polygons[b])//3):
            if polygons[b][dst+offset*2] > 0.3: 
                points_to_draw.append([int(polygons[b][dst]), int(polygons[b][dst+offset])])
        
        points_to_draw = np.asarray(points_to_draw)
        points_to_draw = points_to_draw.astype(np.int32)
        if points_to_draw.shape[0]>0:
            cv2.polylines(img, [points_to_draw],True,f, thickness=2)
            cv2.fillPoly(overlay, [points_to_draw], f)
        
    img = cv2.addWeighted(overlay, 0.4, img, 1 - 0.4, 0)
#     bname = os.path.basename(im)
#     print(bname)
#     cv2.imwrite(os.path.join(out_path, str(imgs)+'.jpg'), img)
    cv2.imwrite(os.path.join(out_path, bname), img)

    
print('total boxes: ', total_boxes)
print('imgs: ', imgs)

901
box:
 [[210.887 494.142 233.885 530.457]]
scores:
 [0.992]
classes:
 [0]
polygons:
 [[496.111 498.275 502.025 504.624 508.042 510.616 513.344 516.136 519.4
  523.212 528.234 528.856 528.73  529.692 528.254 523.625 517.469 513.494
  510.18  506.497 502.208 495.462 495.524 494.995 219.465 216.551 214.172
  213.    212.651 212.008 212.041 212.254 212.59  213.604 215.161 219.503
  223.206 230.213 233.122 235.664 234.377 233.827 233.28  234.698 233.77
  233.135 230.194 223.229   0.932   0.368   0.832   0.444   0.232   0.225
    0.215   0.22    0.353   0.863   0.383   0.957   0.201   0.89    0.637
    0.068   0.017   0.009   0.008   0.017   0.06    0.587   0.833   0.295]]
len box:
 1
len polygons[0]:
 72
box:
 [[192.212 355.554 214.2   389.643]
 [205.702 429.787 211.941 444.166]]
scores:
 [0.996 0.894]
classes:
 [0 0]
polygons:
 [[353.623 355.735 361.938 363.691 367.646 370.68  373.518 376.206 379.429
  383.466 389.93  390.475 390.638 390.339 389.184 382.938 377.104 373.493
  370.141 366

box:
 [[233.292 348.836 256.746 390.18 ]
 [239.936 407.988 257.448 452.381]
 [234.005 448.204 241.003 460.602]]
scores:
 [0.995 0.988 0.938]
classes:
 [0 0 0]
polygons:
 [[351.062 354.186 358.977 361.386 364.888 367.618 370.347 373.41  376.463
  380.77  386.08  388.027 388.031 388.584 384.283 379.952 374.351 370.525
  367.144 363.123 359.378 354.358 350.386 349.53  242.674 239.046 237.229
  235.748 235.113 235.118 234.79  234.61  235.488 236.191 237.501 242.619
  246.825 253.408 255.789 258.164 257.323 257.434 256.525 258.558 256.96
  255.638 253.242 246.932   0.895   0.359   0.725   0.454   0.228   0.279
    0.241   0.267   0.367   0.772   0.472   0.94    0.563   0.87    0.452
    0.082   0.03    0.016   0.015   0.029   0.12    0.628   0.742   0.574]
 [409.762 415.913 420.461 423.884 426.696 428.839 431.098 433.375 436.364
  440.938 444.466 452.836 453.012 451.309 443.559 437.628 433.732 431.019
  428.602 426.025 422.853 415.672 408.82  408.26  245.576 242.215 240.779
  240.484 240.13

 [[231.601 341.025 258.203 386.104]
 [243.797 410.74  260.651 451.421]
 [234.251 448.236 241.359 460.875]]
scores:
 [0.993 0.985 0.967]
classes:
 [0 0 0]
polygons:
 [[345.328 347.601 352.76  355.064 358.735 361.549 364.387 367.669 370.624
  374.497 380.428 382.022 382.084 382.714 378.217 373.317 368.32  364.471
  361.222 357.325 353.576 348.83  343.787 342.937 241.723 237.856 236.019
  234.333 233.608 233.582 233.586 232.953 234.265 235.3   236.644 241.639
  246.182 252.395 255.    256.498 256.33  256.007 255.091 257.142 255.905
  254.424 252.365 246.408   0.816   0.403   0.629   0.489   0.28    0.385
    0.309   0.331   0.394   0.704   0.493   0.876   0.607   0.753   0.535
    0.139   0.051   0.028   0.027   0.047   0.168   0.617   0.618   0.652]
 [411.339 417.173 421.914 425.095 427.68  429.773 432.294 434.665 437.309
  441.989 447.178 455.091 454.485 452.244 444.21  439.748 435.521 432.333
  429.573 426.801 422.768 415.225 410.133 408.769 249.758 246.749 244.964
  244.586 244.258 24

box:
 [[211.047 492.926 234.063 530.479]]
scores:
 [0.989]
classes:
 [0]
polygons:
 [[494.698 497.059 500.871 503.559 507.008 509.605 512.357 515.156 518.551
  522.434 527.887 528.419 528.76  529.795 527.182 522.712 516.666 512.492
  509.153 505.422 501.242 493.799 493.585 493.089 219.469 216.42  214.084
  212.896 212.593 211.842 212.067 212.161 212.454 213.454 214.961 219.409
  223.296 230.477 233.108 235.745 234.812 233.826 233.432 234.869 233.714
  233.472 230.575 223.257   0.933   0.371   0.819   0.409   0.224   0.208
    0.207   0.209   0.33    0.855   0.397   0.958   0.194   0.865   0.669
    0.072   0.017   0.009   0.008   0.016   0.05    0.524   0.847   0.33 ]]
len box:
 1
len polygons[0]:
 72
box:
 [[213.523 459.15  237.639 496.425]]
scores:
 [0.962]
classes:
 [0]
polygons:
 [[459.531 460.447 466.368 468.8   472.539 475.597 478.553 481.421 484.933
  489.334 495.078 496.022 495.48  495.864 493.997 487.631 482.058 478.44
  475.104 471.502 466.672 460.846 459.133 459.725 222.057 

box:
 [[228.064 308.519 263.64  372.411]
 [248.156 416.117 264.725 450.15 ]
 [232.954 446.956 241.67  460.034]
 [226.257 578.615 230.671 586.488]
 [226.513 582.954 230.405 590.124]]
scores:
 [0.984 0.968 0.937 0.329 0.316]
classes:
 [0 0 0 0 0]
polygons:
 [[312.231 313.589 322.754 327.209 333.309 337.626 342.053 346.811 352.722
  360.292 364.742 368.578 372.299 369.233 367.645 355.112 347.063 341.745
  337.968 332.675 323.804 312.973 310.403 310.592 240.982 235.648 231.083
  229.235 229.267 227.798 227.943 228.715 228.851 230.048 235.068 241.343
  247.902 257.97  263.671 263.555 260.85  257.202 258.389 262.18  264.789
  263.548 258.96  247.712   0.906   0.281   0.709   0.502   0.23    0.278
    0.243   0.246   0.341   0.728   0.335   0.917   0.339   0.926   0.523
    0.048   0.019   0.01    0.012   0.022   0.051   0.404   0.876   0.455]
 [417.828 421.198 425.237 426.747 429.719 431.837 434.171 436.444 439.177
  443.009 446.232 450.636 450.687 449.033 446.064 441.217 437.217 434.188
  4

...

box:
 [[196.583  69.159 221.446 134.996]
 [210.256 175.129 271.706 287.315]]
scores:
 [0.997 0.994]
classes:
 [0 0]
polygons:
 [[ 72.544  80.487  89.066  93.241  97.517 100.438 103.531 106.345 110.648
  115.162 123.351 133.278 133.256 132.364 118.457 112.434 107.193 103.37
  100.141  96.34   92.547  85.92   73.961  70.628 206.571 199.779 199.161
  198.005 198.137 198.015 198.12  198.461 198.061 199.258 200.4   206.64
  213.566 219.14  221.357 221.942 221.39  220.795 220.634 222.431 221.42
  221.055 218.64  213.688   0.949   0.515   0.595   0.316   0.224   0.172
    0.194   0.205   0.352   0.57    0.603   0.968   0.926   0.832   0.214
    0.109   0.036   0.021   0.026   0.04    0.102   0.283   0.783   0.902]
 [177.749 180.04  199.681 205.942 219.114 226.979 234.736 243.577 255.296
  266.844 279.118 286.292 289.669 283.378 282.274 259.806 241.655 233.779
  227.51  219.583 205.904 181.316 179.514 174.785 231.172 223.654 213.372
  210.393 211.787 209.883 211.555 211.005 209.723 213.216 221

 [[194.452  44.988 219.473 114.446]
 [215.247 137.866 285.769 264.628]]
scores:
 [0.996 0.816]
classes:
 [0 0]
polygons:
 [[ 47.369  57.445  65.598  69.482  74.149  77.346  80.599  83.194  87.805
   92.844 101.141 111.998 113.643 111.322  96.907  89.002  84.094  80.281
   77.224  73.678  69.651  62.696  49.237  45.731 203.413 196.742 195.896
  193.998 194.393 194.851 195.052 195.86  194.909 195.89  196.685 203.48
  210.672 216.785 219.211 218.465 218.179 217.276 217.607 218.432 218.285
  217.993 216.394 210.962   0.948   0.463   0.61    0.329   0.193   0.204
    0.178   0.194   0.28    0.608   0.569   0.96    0.911   0.838   0.212
    0.082   0.032   0.019   0.022   0.036   0.086   0.249   0.804   0.897]
 [144.623 144.778 167.673 175.946 187.561 196.955 205.453 215.398 226.228
  238.225 260.498 261.392 262.809 260.904 258.159 229.93  213.728 204.176
  197.272 188.892 175.758 146.352 139.544 137.525 241.374 231.786 221.419
  220.098 218.784 218.22  219.265 216.695 218.319 221.718 227.70

 [[189.613   8.839 216.501  68.719]
 [204.172 -11.287 390.637 239.811]
 [334.364 106.469 340.718 111.031]]
scores:
 [0.724 0.384 0.33 ]
classes:
 [0 0 0]
polygons:
 [[ 10.47   17.693  24.484  28.95   33.303  36.418  39.552  42.55   47.242
   51.349  58.092  65.375  66.426  66.82   55.77   48.815  43.249  39.373
   36.194  32.418  28.735  21.247  12.928   9.782 200.228 195.144 192.456
  191.6   191.829 191.565 191.825 192.133 191.42  192.888 195.362 200.49
  206.842 213.387 216.582 216.387 215.277 214.185 214.431 216.332 215.071
  215.797 212.481 206.831   0.846   0.518   0.483   0.319   0.205   0.175
    0.198   0.2     0.374   0.481   0.569   0.909   0.849   0.67    0.276
    0.147   0.057   0.038   0.047   0.079   0.155   0.44    0.556   0.798]
 [-17.389 -11.292   8.585  48.664  78.017 102.496 125.225 153.409 175.423
  209.165 233.99  238.022 237.497 241.819 219.713 164.367 149.495 124.088
  103.147  81.173  64.427  10.538   1.701 -12.453 281.163 245.138 223.915
  214.969 215.562 214

 [0]
polygons:
 [[  3.308   6.085   9.72   13.11   17.537  20.472  23.476  26.319  30.121
   33.467  37.403  39.551  42.081  45.052  40.09   33.778  27.246  23.551
   20.284  16.337  11.91    6.253   2.919   2.332 196.706 192.268 189.485
  188.121 188.347 187.804 187.969 188.852 188.813 190.326 192.075 197.217
  201.085 208.644 212.316 213.301 211.031 211.435 210.073 212.146 211.636
  210.952 207.043 201.256   0.868   0.395   0.751   0.465   0.264   0.298
    0.274   0.288   0.411   0.717   0.516   0.928   0.755   0.884   0.421
    0.106   0.031   0.022   0.023   0.045   0.194   0.727   0.718   0.593]]
len box:
 1
len polygons[0]:
 72
box:
 [[185.862  -0.396 212.597  42.523]]
scores:
 [0.97]
classes:
 [0]
polygons:
 [[  3.211   5.559   9.329  12.547  16.851  19.56   22.577  25.387  28.965
   32.955  37.104  38.275  39.966  41.263  38.396  33.1    26.453  22.667
   19.132  15.371   9.92    5.323   2.391   2.206 196.572 192.777 189.923
  188.654 189.203 187.967 187.455 188.619 189.118 18

IndexError: index 0 is out of bounds for axis 0 with size 0